In [66]:
import numpy as np
import pandas as pd
from pprint import pprint
import scipy.stats as sps

dataset = pd.read_csv('mushrooms_dataset.csv')
print("\n Givebn Mushrooms DAta set:\n\n",dataset)


 Givebn Mushrooms DAta set:

      class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0        p         x           s         n       t    p               f   
1        e         x           s         y       t    a               f   
2        e         b           s         w       t    l               f   
3        p         x           y         w       t    p               f   
4        e         x           s         g       f    n               f   
...    ...       ...         ...       ...     ...  ...             ...   
8119     e         k           s         n       f    n               a   
8120     e         x           s         n       f    n               a   
8121     e         f           s         n       f    n               a   
8122     p         k           y         n       f    y               f   
8123     e         x           s         n       f    n               a   

     gill-spacing gill-size gill-color  ... stalk-surface-below-ring

In [67]:
def id3(data_subset,originaldata,attribute_names,target_attribute_name="class",parent_class = None):
    if len(np.unique(data_subset[target_attribute_name])) <= 1 :
        return np.unique(data_subset[target_attribute_name])[0]
    elif len(data_subset)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[taget_attribute_name],
                                                                                 return_counts = True)[1])]
    elif len(attribute_names)==0:
        return parent_node_class
    else:
        parent_node_class = np.unique(data_subset[target_attribute_name])[np.argmax(np.unique(data_subset[target_attribute_name]
                                                                                              ,return_counts=True)[1])]
        size=int(np.sqrt(len(attribute_names)))
        features = np.random.choice(attribute_names,size,replace=False)
        item_values = [InfoGain(data_subset,feature,target_attribute_name) for feature in features]
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        tree = {best_feature:{}}
        features = [i for i in features if i != best_feature]
        for value in np.unique(data_subset[best_feature]):
            value = value
            data_subset = data_subset.where(data_subset[best_feature] == value).dropna()
            subtree = id3(data_subset,dataset,features,target_attribute_name,parent_node_class)
            tree[best_feature][value] = subtree
        return(tree)
attribute_names = list(dataset.columns)
print("List of attributes: ",attribute_names)
attribute_names.remove('class')
print("Predicting Attributes:",attribute_names)
        

List of attributes:  ['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']
Predicting Attributes: ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']


In [68]:
def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

In [69]:
def InfoGain(data_subset,split_attribute_name,target_name="class"):
    total_entropy = entropy(data_subset[target_name])
    vals,counts = np.unique(data_subset[split_attribute_name],return_counts=True)
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data_subset.where(data_subset[split_attribute_name]==vals[i]).dropna()[target_name])for i in range(len(vals))])
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

In [70]:
def predict(query,tree,default = 'p'):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]]
            except:
                return default
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result

In [71]:
def train_test_split(dataset):
    training_data = dataset.iloc[:round(0.75*len(dataset))].reset_index(drop=True)
    testing_data = dataset.iloc[round(0.75*len(dataset)):].reset_index(drop=True)
    return training_data,testing_data

training_data = train_test_split(dataset)[0]
testing_data = train_test_split(dataset)[1]

In [72]:
def RandomForest_Train(dataset,number_of_Trees):
    random_forest_sub_tree = []
    for i in range(number_of_Trees):
        bootstrap_sample = dataset.sample(frac=1,replace=True)
        bootstrap_training_data = train_test_split(bootstrap_sample)[0].any()
        bootstrap_testing_data = train_test_split(bootstrap_sample)[1].any()
        target_attribute_name = bootstrap_training_data.drop(labels=['class'])
    random_forest_sub_tree.append(id3(bootstrap_training_data,bootstrap_training_data,attribute_names,target_attribute_name,
                                     parent_node_class))
    return random_forest_sub_tree
    random_forest = RandomForest_Train(dataset,50)

In [73]:
def RandomForest_Predict(query,random_forest,default='p'):
    predictions = []
    for tree in random_forest:
        predictions.append(predict(query,tree,default))
    return sps.mode(predictions)[0][0]
    predictions = RandomForest_Predict(query,random_forest)
    print('prediction:',prediction)
query = testing_data.iloc[0,:].drop('class').to_dict()
query_target = testing_data.iloc[0,0]
print('class',query_target)

class p


In [74]:
def RandomForest_Test(testing_data,random_forest):
    for i in range(len(testing_data)):
        testing_data['predictions'] = None
        query = testing_data.iloc[i,:].drop('class').to_dict()
        testing_data.iloc[i,'predictions'] = RandomForest_Predict(query,random_forest,default='p')
        accuracy=sum(testing_data['predictions'] == testing_data['class'])/len(testing_data)*100
        pprint('The Prediction accuracy is:',sum(testing_data['predictions']))
        return accuracy
        RandomForest_Test(testing_data,random_forest)

In [75]:
tree = id3(dataset,dataset,attribute_names,"class")
pprint(tree

IndexError: index 0 is out of bounds for axis 0 with size 0